需求不定的运输问题

|      | B1  | B2  | B3  | B4  | 产量  |
| ---- | --- | --- | --- | --- | --- |
| A1   | 16  | 13  | 22  | 17  | 50  |
| A2   | 14  | 13  | 19  | 15  | 60  |
| A3   | 19  | 20  | 23  | -   | 50  |
| 最低需求 | 30  | 70  | 0   | 10  |     |
| 最高需求 | 50  | 70  | 30  | inf |     |

In [3]:
import gurobipy as gp
from gurobipy import GRB

# 数据定义
costs = {
    ('A1', 'B1'): 16, ('A1', 'B2'): 13, ('A1', 'B3'): 22, ('A1', 'B4'): 17,
    ('A2', 'B1'): 14, ('A2', 'B2'): 13, ('A2', 'B3'): 19, ('A2', 'B4'): 15,
    ('A3', 'B1'): 19, ('A3', 'B2'): 20, ('A3', 'B3'): 23, ('A3', 'B4'): GRB.INFINITY
}

supply = {
    'A1': 50,
    'A2': 60,
    'A3': 50
}

min_demand = {
    'B1': 30,
    'B2': 70,
    'B3': 0,
    'B4': 10
}

max_demand = {
    'B1': 50,
    'B2': 70,
    'B3': 30,
    'B4': GRB.INFINITY
}

In [4]:
# 创建模型
model = gp.Model("Transportation_Problem")

# 添加变量
x = model.addVars(costs.keys(), vtype=GRB.CONTINUOUS, name="x")

# 设置目标函数
model.setObjective(gp.quicksum(x[i, j] * costs[i, j] for i, j in costs if costs[i, j]), GRB.MINIMIZE)

# 添加约束条件
model.addConstrs((gp.quicksum(x[i, j] for j in min_demand) == supply[i] for i in supply), name="Supply")
model.addConstrs((gp.quicksum(x[i, j] for i in supply) >= min_demand[j] for j in min_demand), name="MinDemand")
model.addConstrs((gp.quicksum(x[i, j] for i in supply) <= max_demand[j] for j in max_demand), name="MaxDemand")

# 求解模型
model.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-07
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 7840HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 11 rows, 12 columns and 36 nonzeros
Model fingerprint: 0x8dd4c1ce
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 1e+100]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+100]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 5 rows and 2 columns
Presolve time: 0.02s
Presolved: 6 rows, 11 columns, 20 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.3800000e+03   1.250000e+01   0.000000e+00      0s
       3    2.4600000e+03   0.000000e+00   0.000000e+00      0s


In [5]:
# 输出结果
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for v in model.getVars():
        if v.X > 0:
            print(f"{v.VarName} = {v.X}")
    print(f"Total Cost: {model.ObjVal}")
else:
    print("No optimal solution found.")


Optimal solution found:
x[A1,B2] = 50.0
x[A2,B2] = 20.0
x[A2,B4] = 40.0
x[A3,B1] = 50.0
Total Cost: 2460.0
